1フレームあたりのサンプリング手法の比較


ランダム、FPS、提案手法で比較

手順
1. データセットの準備
2. データセットから1フレーム分のデータを取得
3. ダウンサンプリングインデックスを読み込み
4. ダウンサンプリング
5. アップサンプリング
6. テスト実行

パラメータ
-テストデータのインデックス
-サンプリング率

0-100: Airplane

491-590: Car

1509-1528: Person

751-836: Desk

In [1]:
import random
import pandas as pd
import numpy as np
# import datetime

from copy import deepcopy

from pathlib import Path
import sys
import importlib

from easydict import EasyDict
from pprint import pprint
import torch
import os
import time

import time
from tqdm import tqdm
import datetime

In [2]:
BASE_DIR = Path('../Pointnet_Pointnet2_pytorch')
sys.path.append(str(BASE_DIR))
sys.path.append(str(BASE_DIR/'models'))
print(sys.path)

from data_utils.ModelNetDataLoader import ModelNetDataLoader
from pointnet2_utils import farthest_point_sample

RESULT_DIR = BASE_DIR/'../Mywork/Results'
FIGURE_DIR = RESULT_DIR/'Figures'
FILE_DIR = RESULT_DIR/'Files'

['/mnt/workspace/Github/Share/Mywork', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/tmp/OpenPCDet', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '../Pointnet_Pointnet2_pytorch', '../Pointnet_Pointnet2_pytorch/models']


In [3]:
#GA_downsampling用のargs
args = EasyDict({
    'use_cpu': 'False',
    'gpu': '0',
    'batch_size': 103, 
    'num_category': 40,
    'num_point': 1024,
    'log_dir': 'pointnet2_cls_ssg',
    'use_normals': False, #bestを使うときはTrue
    'use_uniform_sample': False, #Trueにするとデータロード時にFPS実行。FPSには初期値にランダム性あり。TrueとFalseが逆な気がする
    'num_votes': 3,
    'output': FILE_DIR,
    })

num_class = args.num_category
print("The num of class is {0}".format(num_class))
print(args.log_dir)

The num of class is 40
pointnet2_cls_ssg


In [4]:
import DataLoaderForGAdownsampling

data_path = BASE_DIR / Path('data/modelnet40_normal_resampled/')
fv_path = FILE_DIR/'seed0'/'Feature_vector_base'/'2021-12-07_13-27.csv'
# fv_path = FILE_DIR/'seed0'/'Feature_vector_base'/'average2021-12-16_22-54.csv'

GA_dataset = DataLoaderForGAdownsampling.FVDataLoader(root=data_path, fv_path=fv_path, args=args, split='test', process_data=False)
GADataLoader = torch.utils.data.DataLoader(GA_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

The size of test data is 2468


In [5]:
model_name = "cls_ssg_feature_selection" #feature_selectionっていう名前だけど、実態は２つに分けたもの
model = importlib.import_module(model_name)

Feature_vector_calculator = model.get_first_model(num_class, normal_channel=args.use_normals)
Feature_vector_calculator = Feature_vector_calculator.cuda()
Classifier_from_feature_vector = model.get_second_model(num_class, normal_channel=args.use_normals)

experiment_dir = BASE_DIR / Path('log/classification/') / Path(args.log_dir)
checkpoint = torch.load(str(experiment_dir) + '/checkpoints/best_model.pth') #学習済みデータを使う

Feature_vector_calculator.load_state_dict(checkpoint['model_state_dict'])
Classifier_from_feature_vector.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [423]:
# import requests
# import json

# def slack_notify(message):
#     webhook_url = ''

#     text = message

#     requests.post(webhook_url, data = json.dumps({
#         "text": text
#     }));
    
# # slack_notify('test')

In [6]:
rate_list = [1/2**k for k in range(6)]
print(rate_list)

[1.0, 0.5, 0.25, 0.125, 0.0625, 0.03125]


In [189]:
print(GA_dataset.npoints)

1024


In [7]:
def initialize(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [8]:
class Sampling_test:
#     rate_list = [1/2**k for k in range(6)]
    def __init__(self, dataset, model1, model2, rate_list, sampling_iteration, predict_iteration):
        self.dataset = dataset
        self.npoints = dataset.npoints
        self.Feature_vector_calculator = model1.eval().cpu()
        self.Classifier_from_feature_vector = model2.eval().cpu()
        self.sampling_iteration = sampling_iteration
        self.predict_iteration = predict_iteration
        
        self.sampling_list = []
        self.rate_list = rate_list

    def repeat_test(self, point, target):
        mean_correct = []
        point = point.cpu()
        point = point.transpose(2, 1)
        with torch.no_grad():
            for i in range(self.predict_iteration):        
                feature_vector = self.Feature_vector_calculator(point)
                pred, _ = self.Classifier_from_feature_vector(feature_vector)
                pred_choice = pred.data.max(1)[1]
                correct = pred_choice.eq(target).sum()
                mean_correct.append(correct.item() / float(point.size()[0]))
        instance_acc = np.mean(mean_correct)
        return instance_acc
    
    def set_random_list(self, rate_list):
        N = self.npoints
        random_list = np.empty(0)
        for rate in self.rate_list:
            select_size = int(N * rate)
            for _ in range(self.sampling_iteration):
                bit_list = np.zeros(N)
                bit_list[random.sample(range(N), select_size)] = 1
                random_list = np.append(random_list, bit_list)

        random_list = random_list.reshape(len(rate_list), self.sampling_iteration, N)
        self.sampling_list = random_list
        self.rate_list = rate_list
        return None
    
    def set_fps_list(self, rate_list, data_idx):
        N = self.npoints
        data, _, _ = self.dataset.__getitem__(data_idx)
        data = torch.from_numpy(data.astype(np.float32)).clone().view(1, 1024, 3)
        fps_list = np.empty(0)
        for rate in rate_list:
            select_size = int(N * rate)
            for _ in range(self.sampling_iteration):
                bit_list = np.zeros(N)
                sample_idx = farthest_point_sample(data, select_size)
                bit_list[sample_idx] = 1
                fps_list = np.append(fps_list, bit_list)
            
        fps_list = fps_list.reshape(len(rate_list), self.sampling_iteration, N)
        self.sampling_list = fps_list
        self.rate_list = rate_list
        return None
    
    def set_sampling_list(self, sampling_list, rate_list):
        self.sampling_list = sampling_list
        self.rate_list = rate_list
        self.sampling_iteration = sampling_list.shape[1]
        return None
    
    def sampling_test(self, data_idx, method):
        acc_arr = np.empty(0)
        original_points, _, target = self.dataset.__getitem__(data_idx)
        original_points = torch.from_numpy(original_points.astype(np.float32)).clone().view(1, self.npoints, 3)
        _, N, _ = original_points.shape
        if method == "random":
            self.set_random_list(self.rate_list)
#             sampling_list = self.sampling_list
        elif method == "fps":
            self.set_fps_list(self.rate_list, data_idx)
#             sampling_list = self.sampling_list
        for k in tqdm(range(len(self.rate_list)), total=len(self.rate_list)):
            pop = self.sampling_list[k]
            for individual in pop:
                individual = np.where(individual == 1)
                individual = individual[0]
                sampled_points = original_points[:, individual, :] #選択インデックスの点群を削除 1024->512
                sampled_points = torch.tile(sampled_points, (1, int(N/len(individual)), 1)) #アップサンプリングしてMLに入力 512->1024
                acc = self.repeat_test(sampled_points, target)
                acc_arr = np.append(acc_arr, acc)
        
        acc_arr = acc_arr.reshape(len(self.rate_list), self.sampling_iteration)
        print(acc_arr.shape)
        acc_arr = np.mean(acc_arr, axis=1)
        return acc_arr

In [9]:
rate_list = [1/2**k for k in range(6)]
random_env = Sampling_test(GA_dataset, Feature_vector_calculator, Classifier_from_feature_vector, \
                             rate_list, sampling_iteration=30, predict_iteration=50)
ga_env = Sampling_test(GA_dataset, Feature_vector_calculator, Classifier_from_feature_vector, \
                             rate_list, sampling_iteration=1, predict_iteration=50)

### GAダウンサンプリング（提案手法）のテストを実行

In [10]:
# pprint(os.listdir(FILE_DIR/'seed0'/'FeatureSelection_toZero_GA'/'log'))
data_idx = 615
pop_list = os.listdir(FILE_DIR/'seed0'/'GA_downsampling'/'pop')
pop_list = [s for s in pop_list if 'data'+str(data_idx) in s]
pop_list.sort()
pop_list = pop_list[::-1]
pprint(pop_list)

['data615_ngen1000_rate0.5_seed0_2022-01-02_15-03.csv',
 'data615_ngen1000_rate0.25_seed0_2022-01-03_04-49.csv',
 'data615_ngen1000_rate0.125_seed0_2022-01-03_18-19.csv',
 'data615_ngen1000_rate0.0625_seed0_2022-01-04_07-36.csv',
 'data615_ngen1000_rate0.03125_seed0_2022-01-04_20-40.csv']


In [11]:
#ga_sample_list作成
N = 1024
pop_size = 30
ga_sample_list = np.ones(N*pop_size).reshape(1, pop_size, N)
for select_pop in pop_list:
    pop_df = pd.read_csv(FILE_DIR/'seed0'/'GA_downsampling'/'pop'/select_pop)
    for pop_idx in pop_df.values:
        value = np.zeros(N)
        value[pop_idx] = 1
        ga_sample_list = np.append(ga_sample_list, value)
ga_sample_list = ga_sample_list.reshape(len(rate_list), pop_df.index.size, N)
pprint(ga_sample_list.shape)

(6, 30, 1024)


In [12]:
seed = 0
initialize(seed)
ga_env.set_sampling_list(ga_sample_list, rate_list)
ga_acc_arr = ga_env.sampling_test(data_idx, method="ga")
print(ga_acc_arr)

  0%|                                                                                                | 0/6 [00:04<?, ?it/s]


KeyboardInterrupt: 

### ランダムとFPSのテストを実行

In [13]:
data_idx = 615
seed = 0
initialize(seed)
fps_acc_arr = random_env.sampling_test(data_idx, method="fps")
print(fps_acc_arr)
random_acc_arr = random_env.sampling_test(data_idx, method="random")
print(random_acc_arr)

 33%|█████████████████████████████                                                          | 2/6 [11:34<23:08, 347.14s/it]


KeyboardInterrupt: 

### 結果の保存

In [428]:
result = np.empty(len(rate_list))
result = np.vstack([random_acc_arr, fps_acc_arr, ga_acc_arr])
print(result)
result_df = pd.DataFrame(data=result)

[[0.999      0.84366667 0.50366667 0.1        0.13333333 0.        ]
 [0.99933333 0.99933333 0.89566667 0.3        0.13333333 0.        ]
 [0.99933333 0.99933333 0.99433333 1.         0.86666667 0.73333333]]


In [429]:
method_name = 'downsampling/' #何の結果か、どのような加工を加えているか、何個ずつか
# prefix = 'ngen{0}_rate{1}_seed{}_'.format(ngen, select_size, seed)
prefix = 'minsize{}_data{}_'.format(int(rate_list[-1]*1024), data_idx)
ext = ".csv"
df = result_df
#-------------------------------------
save_dir = FILE_DIR / Path(method_name) 
# print(save_dir)
save_dir.mkdir(exist_ok=True, parents=True)
timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
# print(timestr)
file_name = prefix + timestr + ext
df.to_csv(save_dir/file_name, index=False)
print("Saved in:", save_dir/file_name)
slack_notify("Saved in:"+ str(save_dir/file_name))

Saved in: /mnt/workspace/Github/Pointnet_Pointnet2_pytorch/Mywork/Results/Files/downsampling/minsize32_data615_2022-01-05_12-29.csv


# ここから余分

# 点群の可視化

In [6]:
from Viewer_oka import CreateRGBVectorFromRGB, ViewColorPoints 

(5, 1)
<class 'numpy.ndarray'>
(5, 3)


In [20]:
N = 1024
rgb = CreateRGBVectorFromRGB(N).reshape(-1, 1)
print(rgb.shape)

data_idx = 615
points, _, _ = GA_dataset.__getitem__(data_idx)
print(points.shape)

(1024, 1)
(1024, 3)


In [21]:
ViewColorPoints(points, rgb)

<class 'numpy.ndarray'>
(1024, 3)


In [257]:
fps_acc_arr[1] = 0.5
print(fps_acc_arr)
print(np.sort(fps_acc_arr)[::-1])
print(all(fps_acc_arr == np.sort(fps_acc_arr)[::-1]))

[1.  0.5 1.  1.  0.  0. ]
[1.  1.  1.  0.5 0.  0. ]
False


In [262]:
ideal_list = []
seed = 0
for idx in check_list:
    initialize(seed)
    data_idx = idx
    random_acc_arr = test_env.sampling_test(data_idx, method="random")
    fps_acc_arr = test_env.sampling_test(data_idx, method="fps")
    print(data_idx, random_acc_arr)
    print(data_idx, fps_acc_arr)
    if all(fps_acc_arr == np.sort(fps_acc_arr)[::-1]) or all(random_acc_arr == np.sort(random_acc_arr)[::-1]):
        ideal_list.append(data_idx)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:14<00:00,  1.35s/it]


718 [0.12 0.08 0.14 0.11 0.   0.  ]
718 [0.1  0.03 0.05 0.   0.   0.  ]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:12<00:00,  1.33s/it]


1711 [0.05 0.07 0.06 0.01 0.   0.06]
1711 [0.02 0.23 0.04 0.   0.   0.16]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:14<00:00,  1.35s/it]


...

2188 [0.72 0.96 0.92 0.65 0.05 0.  ]
2188 [0.83 0.93 0.89 0.45 0.   0.  ]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:12<00:00,  1.33s/it]


1980 [0.87 0.75 0.49 0.26 0.16 0.07]
1980 [0.74 0.65 0.47 0.19 0.   0.  ]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:14<00:00,  1.34s/it]

2403 [0.81 0.43 0.25 0.49 0.62 0.15]
2403 [0.79 0.91 0.4  0.84 0.   0.  ]


In [263]:
car_list = range(490, 590)
car_ideal_list = []
seed = 0
for idx in car_list:
    initialize(seed)
    data_idx = idx
    random_acc_arr = test_env.sampling_test(data_idx, method="random")
    fps_acc_arr = test_env.sampling_test(data_idx, method="fps")
    print(data_idx, random_acc_arr)
    print(data_idx, fps_acc_arr)
    if all(fps_acc_arr == np.sort(fps_acc_arr)[::-1]) or all(random_acc_arr == np.sort(random_acc_arr)[::-1]):
        car_ideal_list.append(data_idx)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:12<00:00,  1.33s/it]


490 [1.   1.   1.   0.96 0.1  0.  ]
490 [1. 1. 1. 1. 0. 0.]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:12<00:00,  1.33s/it]


491 [1.   1.   1.   0.99 0.57 0.  ]
491 [1. 1. 1. 1. 0. 0.]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:12<00:00,  1.32s/it]


...

587 [1.   1.   1.   1.   0.72 0.  ]
587 [1. 1. 1. 1. 0. 0.]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:11<00:00,  1.32s/it]


588 [1.   1.   1.   1.   0.31 0.  ]
588 [1. 1. 1. 1. 0. 0.]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:12<00:00,  1.32s/it]

589 [1.   1.   1.   1.   0.48 0.  ]
589 [1. 1. 1. 1. 0. 0.]


In [264]:
person_list = range(1508, 1528)
person_ideal_list = []
seed = 0
for idx in person_list:
    initialize(seed)
    data_idx = idx
    random_acc_arr = test_env.sampling_test(data_idx, method="random")
    fps_acc_arr = test_env.sampling_test(data_idx, method="fps")
    print(data_idx, random_acc_arr)
    print(data_idx, fps_acc_arr)
    if all(fps_acc_arr == np.sort(fps_acc_arr)[::-1]) or all(random_acc_arr == np.sort(random_acc_arr)[::-1]):
        person_ideal_list.append(data_idx)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:11<00:00,  1.31s/it]


1508 [1.   1.   1.   1.   0.94 0.69]
1508 [1.   1.   1.   1.   0.95 0.48]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:11<00:00,  1.31s/it]


1509 [1.   1.   1.   0.97 0.59 0.42]
1509 [1.   1.   1.   1.   0.28 1.  ]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:10<00:00,  1.31s/it]


...

1525 [1.   1.   1.   1.   0.94 0.64]
1525 [1.   1.   1.   1.   1.   0.75]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:08<00:00,  1.29s/it]


1526 [1. 1. 1. 1. 1. 1.]
1526 [1. 1. 1. 1. 1. 1.]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:11<00:00,  1.32s/it]

1527 [1.   1.   1.   0.99 0.65 0.27]
1527 [1.   1.   1.   1.   0.67 0.04]


In [265]:
chair_list = range(590, 690)
chair_ideal_list = []
seed = 0
for idx in chair_list:
    initialize(seed)
    data_idx = idx
    random_acc_arr = test_env.sampling_test(data_idx, method="random")
    fps_acc_arr = test_env.sampling_test(data_idx, method="fps")
    print(data_idx, random_acc_arr)
    print(data_idx, fps_acc_arr)
    if all(fps_acc_arr == np.sort(fps_acc_arr)[::-1]) or all(random_acc_arr == np.sort(random_acc_arr)[::-1]):
        chair_ideal_list.append(data_idx)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:12<00:00,  1.33s/it]


590 [1.   1.   1.   1.   1.   0.97]
590 [1. 1. 1. 1. 1. 0.]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:14<00:00,  1.34s/it]


591 [1.   1.   1.   0.99 0.82 0.37]
591 [1.   1.   1.   1.   0.97 0.  ]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:13<00:00,  1.33s/it]


...

687 [1.   1.   1.   1.   1.   0.85]
687 [1.   1.   1.   1.   0.99 0.  ]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:12<00:00,  1.32s/it]


688 [1.   1.   1.   1.   1.   0.65]
688 [1. 1. 1. 1. 1. 0.]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:13<00:00,  1.34s/it]

689 [1.   1.   1.   1.   1.   0.65]
689 [1. 1. 1. 1. 1. 0.]


In [267]:
print(ideal_list)

print(car_ideal_list)

print(person_ideal_list)

print(chair_ideal_list)

[1790, 1491, 1478, 2242, 2270, 1431, 1270, 1033, 1535, 1980]
[490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589]
[1508, 1509, 1510, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527]
[590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 651, 652, 653, 654, 655, 656, 657, 658

In [ ]:
result = np.empty(0)
result = np.append(result, mean_correct)
result = np.append(result, mean_dist)
result = result.reshape(2, 10)
print(result)

In [ ]:
result_df = pd.DataFrame(data=result)
pprint(resdult_df)

In [186]:
sl = slice(160, 220)
each_acc = pd.read_csv(FILE_DIR/"seed0/each_frame_accuracy/2021-12-15_12-06.csv")
each_acc = each_acc.values.reshape(-1)
print(each_acc.shape)
print(np.sort(each_acc)[sl])
print(each_acc.argsort()[sl])
# print(each_acc[1498])
check_list = each_acc.argsort()[sl]

(2468,)
[0.13 0.13 0.16 0.19 0.2  0.21 0.22 0.24 0.27 0.3  0.32 0.35 0.35 0.35
 0.36 0.38 0.38 0.4  0.41 0.42 0.45 0.45 0.49 0.49 0.5  0.5  0.5  0.5
 0.51 0.54 0.54 0.54 0.55 0.57 0.57 0.6  0.61 0.63 0.65 0.65 0.65 0.68
 0.71 0.71 0.72 0.72 0.72 0.72 0.72 0.73 0.73 0.74 0.75 0.76 0.77 0.78
 0.78 0.78 0.79 0.79]
[ 718 1711 2437 2102  890  487 2013 1869  286 1746 1629  932 1790  905
  620 1433 1687 2045 1633  740 1457  938  854 2464  901 2229 1605 1654
 2060 1491 1442 2397 1478  110 2242 2066 2270 1431  863 2459  909 2301
 2409  408 1377  815  888 1270  944 2096 1678 1033 1668 1356 2255 1686
 1535 2188 1980 2403]


In [156]:
each_acc = pd.read_csv(FILE_DIR/"seed0/each_frame_accuracy/2021-12-15_12-06.csv")
each_acc = each_acc.values.reshape(-1)
# each_acc = each_acc[566]
print(each_acc[650])
print(each_acc[1512])
# print(each_acc.shape)
# print(np.sort(each_acc))
# print(each_acc.argsort())

0.1
0.01


In [ ]:
data_idx = 500
mean_correct = np.empty(0)
# mean_dist = np.empty(0)

Classifier_from_feature_vector = Classifier_from_feature_vector.eval().cpu()
Feature_vector_calculator = Feature_vector_calculator.eval().cpu()

for k in tqdm(range(10), total=9):
    miss = np.empty(0)
    dist_arr = np.empty(0)
    rate = 1/ (2**k)
    
    original_points, target_fv, target = GA_dataset.__getitem__(data_idx)
    target_fv = torch.from_numpy(target_fv.astype(np.float32)).clone().view(1, -1)
    # print("{}{: ^7}{}{: ^7}{}{: ^7}{}{: ^7}{}".format('|','Dist','|', 'Ans','|', 'Pred', '|', 'Correct', '|'))
    # print("---------------------------------")

    for i in range(1000):
        initialize(i)

        with torch.no_grad():
            points = random_down_up(original_points, rate)
            points = torch.from_numpy(points.astype(np.float32)).clone().view(1, 1024, 3).transpose(1, 2)
            

            feature_vector = Feature_vector_calculator(points)
            dist = torch.sum(torch.cdist(feature_vector.view(1, 1, -1), target_fv.view(1, 1, -1)), dim=0)
        #     score = torch.cat((score, dist[0]), 0)
        #     print("Distance of feature vector: ", dist[0])

            pred, _ = Classifier_from_feature_vector(feature_vector)
            pred_choice = pred.data.max(1)[1].detach().numpy().astype(np.int32)



    #     print("{}{: ^.5f}{}{: ^7}{}{: ^7}{}{: ^7}{}".format('|', float(dist[0]),'|', str(target),'|', str(pred_choice[0]), '|', target == pred_choice[0], '|'))  
    #     print("Dist: {0}  Answer: {1}   Pred: {1}   {2}".format(target, pred_choice[0], target == pred_choice))
        if target != pred_choice[0]:
            miss = np.append(miss, 0)
        else:
            miss = np.append(miss, 1)
        dist_arr = np.append(dist_arr, dist)
    mean_correct = np.append(mean_correct, np.mean(miss))
    mean_dist = np.append(mean_dist, np.mean(dist_arr))
        
    
    # if miss > 0:
    #     print("{} : {}".format(data_idx, miss))

In [ ]:
def random_down_up(points, rate):
    N, _ = points.shape
    select_idx = random.sample(range(N), int(N*rate))
    points = points[select_idx, :]
    points = np.tile(points, (int(1/rate), 1))
    
    return points

In [181]:
N = 1024
random_list = np.empty(0)
for i in range(6):
    bit_list = np.zeros(N)
    select_size = int(1024/ (2**i))
    bit_list[random.sample(range(N), select_size)] = 1
    random_list = np.append(random_list, bit_list)
    
random_list = random_list.reshape(6, N)
print(random_list)
print(np.sum(random_list, axis=1))

[[1. 1. 1. ... 1. 1. 1.]
 [1. 0. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[1024.  512.  256.  128.   64.   32.]


In [365]:
# pprint(os.listdir(FILE_DIR/'seed0'/'FeatureSelection_toZero_GA'/'log'))
hof_list = os.listdir(FILE_DIR/'seed0'/'GA_downsampling'/'Hall of fame')
hof_list = [s for s in hof_list if 'data1980' in s]
hof_list.sort()
hof_list = hof_list[::-1]
pprint(hof_list)

['data1980_ngen1000_rate0.5_seed0_2021-12-25_06-51.csv',
 'data1980_ngen1000_rate0.25_seed0_2021-12-25_20-36.csv',
 'data1980_ngen1000_rate0.125_seed0_2021-12-26_09-55.csv',
 'data1980_ngen1000_rate0.0625_seed0_2021-12-26_23-22.csv',
 'data1980_ngen1000_rate0.03125_seed0_2021-12-27_12-31.csv']


In [367]:
ga_sample_list = np.ones(1024).reshape(1, 1, -1)
# print(ga_sample_list)
for select_hof in hof_list:
    hof_df = pd.read_csv(FILE_DIR/'seed0'/'GA_downsampling'/'Hall of fame'/select_hof)
    
    new_stack = np.empty(0)
    for value in hof_df.values:
        for ga_sample in ga_sample_list:
            ga_sample = np.vstack([ga_sample, value])
            new_stack = np.append(new_stack, ga_sample)
    ga_sample_list = new_stack.reshape(len(hof_df)*len(ga_sample_list), -1, 1024)
    
pprint(ga_sample_list)
print(ga_sample_list.shape)

array([[[1., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 0., 1., 0.],
        [0., 0., 0., ..., 1., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 0., 1., 0.],
        [0., 0., 0., ..., 1., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 0., 1., 0.],
        [0., 0., 0., ..., 1., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 0., 1., 0.],
        [0., 0., 0., ..., 1., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
(4, 6, 1024)
